In [ ]:
from aleph_alpha_client import Client, Prompt, CompletionRequest, CompletionResponse, SemanticEmbeddingRequest, SemanticEmbeddingResponse, SemanticRepresentation
from scipy import spatial
import numpy as np
import os
from dotenv import load_dotenv

In [ ]:
load_dotenv()

client = Client(token=os.getenv("AA_TOKEN"))

### Step 1: Let's do a really simple QA prompt
In this step we will ask luminous a question from the manual. Let's see how it answers.
Write a completionrequest and make luminous answer it. Don't forget the stop sequences.
You can find the documentation for the completionrequest here: https://docs.aleph-alpha.com/docs/tasks/complete/

In [ ]:
prompt = "Q: How is Stop Category 0 defined?\nA:"

# TODO: Write the completion request
request = None

# TODO: Call the client to run the completionrequest
response = None
print(response.completions[0].completion)

##### Solution

```
prompt = "Q: How is Stop Category 0 defined?\nA:"

request = CompletionRequest(
    prompt=Prompt.from_text(prompt),
    maximum_tokens=64,
    stop_sequences=["Q:", "Q"],
)

response = client.complete(request = request, model = "luminous-extended")
print(response.completions[0].completion)
```

#### Step 2: Let's do a QA prompt with a context
Obviously, we can't expect luminous to answer the question if we don't give it any context. Let's see how it answers when we give it some context.
Write the new prompt and make luminous answer it. Don't forget the stop sequences.

In [ ]:
context = """The robot has protective and emergency stop functions (stop category 0 or 1, in accordance with IEC 60204-1).

| Stop category 0 | As defined in IEC 60204-1, stopping by immediate removal of power to the machine actuators. |  
|---|---|  
| Stop category 1 | As defined in IEC 60204-1, a controlled stop with power avail- able to the machine actuators to achieve the stop and then re- moval of power when the stop is achieved. |  """

# TODO write a prompt that makes luminous answer the question based on the context
prompt = f"Prompt that includes the {context}"

request = None

response = None
print(response.completions[0].completion)



##### Solution

```
context = """he robot has protective and emergency stop functions (stop category 0 or 1, in accordance with IEC 60204-1).

| Stop category 0 | As defined in IEC 60204-1, stopping by immediate removal of power to the machine actuators. |  
|---|---|  
| Stop category 1 | As defined in IEC 60204-1, a controlled stop with power avail- able to the machine actuators to achieve the stop and then re- moval of power when the stop is achieved. |  """

# TODO write a prompt that makes luminous answer the question based on the context
prompt = f"This system answers questions about the robot based on the context. \n\nContext: {context}\nQ: How is Stop Category 0 defined?\nA:"

request = CompletionRequest(
    prompt=Prompt.from_text(prompt),
    maximum_tokens=64,
    stop_sequences=["Q:", "Q"],
)

response = client.complete(request = request, model = "luminous-extended")
print(response.completions[0].completion)

```

#### Step 3: Let's use our search we coded in the previous notebook to find the relevant context for the question and then use that context to answer the question
This step is a bit long, but it is a good exercise to see how we can use the search we coded in the previous notebook to find the relevant context for the question and then use that context to answer the question.

In [ ]:
texts = [
    "Pinocchio was not a boy, but a wooden puppet. He was made by a carpenter named Geppetto. He was a very naughty puppet. He was always getting into trouble. He was always lying.",
    "Most commonly associated with the polar regions, permafrost is soil and rocky material that stays frozen continuously for at least two years. Normally it lies beneath an active layer that melts and freezes depending on the season. Less well known is that permafrost can also be found on steep mountain walls.",
    "Toheroa are a clam that grow as large as a human hand and burrow in intertidal sands on just a handful of epic surf-swept beaches – mainly on the west coast of New Zealand's North Island, but also in isolated colonies at places like Oreti, a beach at the nation's southern tip.",
    "On the neighbourhood's southern edge, cutting through Queens like a backbone, is Roosevelt Avenue. Here, conversations don't stop when the 7 train rattles overhead, they just get louder. Phone repair shops run by Tibetans with makeshift shrines displayed between plastic iPhone covers abut Latin American bakeries churning out pillowy almojábanas (Colombian cheese bread) and crispy empanadas."
]
        
question = "What city is located at the river Neckar?"

#TODO: embed the contexts
embedded_texts = []
for text in texts:
    pass

# TODO: embed the question
embedded_question = None

# TODO: calculate the cosine similarity between the question and the contexts
similarities = []
for embedded_text in embedded_texts:
    pass

# Get the text in the most similar context
selected_context = texts[np.argmax(similarities)]

# TODO: write a prompt that makes luminous answer the question based on the context
prompt = f"Another Prompt"

# TODO: write the CompletionRequest
request = None

response = None

print(response.completions[0].completion)
print(selected_context)


##### Solution


```
splitted_context = open("sample_file1.md").read().split("######")
# remove small splits
splits = []
for split in splitted_context:
    if len(split) > 30:
        splits.append(split)
        
question = "How is Stop Category 0 defined?"

#TODO: embed the contexts
embedded_splits = []
for split in splits:
    embedded_splits.append(client.semantic_embed(SemanticEmbeddingRequest(prompt=Prompt.from_text(split), representation=SemanticRepresentation.Document, compress_to_size=128), model="luminous-base").embedding)

# TODO: embed the question
embedded_question = client.semantic_embed(SemanticEmbeddingRequest(prompt=Prompt.from_text(question), representation=SemanticRepresentation.Document, compress_to_size=128), model="luminous-base").embedding

# TODO: calculate the cosine similarity between the question and the contexts
similarities = []
for embedded_split in embedded_splits:
    similarities.append(1 - spatial.distance.cosine(embedded_question, embedded_split))

# Get the text in the most similar context
selected_context = splits[np.argmax(similarities)]

# TODO: write a prompt that makes luminous answer the question based on the context
prompt = f"This system answers questions about the robot based on the context. \n\nContext: {context}\nQ: How is Stop Category 0 defined?\nA:"

# TODO: write the CompletionRequest
request = CompletionRequest(
    prompt = Prompt.from_text(prompt),
    maximum_tokens = 32,
    stop_sequences=["\n\n", "Q:", "Q"],
)

response = client.complete(request = request, model = "luminous-extended")

print(response.completions[0].completion)


```